In [56]:
import splusdata
from tqdm import tqdm

from astropy.io import fits

from auxiliar import *

In [26]:
conn = splusdata.Core()

In [27]:
df = conn.query(f"""
        SELECT 
        TOP 2000
        det.id, det.ra, det.dec, det.field, det.B, det.A, det.flux_radius_50, det.theta, det.fwhm,
        
        r.r_auto
        FROM "idr4_dual"."idr4_detection_image" AS det 
        
        LEFT OUTER JOIN "idr4_dual"."idr4_dual_r" AS r ON r.id = det.id
        
        WHERE 1 = CONTAINS( POINT('ICRS', det.ra, det.dec), 
				CIRCLE('ICRS', 0.5, 0.4, 1) ) 
				AND r.r_auto < 21
        """)

df = df.to_pandas()

In [52]:
df.to_csv('data.csv', index=False)

In [37]:
stamps = []

for key, value in tqdm(df.iterrows()):
    stamp = conn.stamp(value["RA"], value["DEC"], 250, "R")
    stamps.append(stamp)

2000it [24:46,  1.35it/s]


In [51]:
for key, value in tqdm(df.iterrows()):
    stamps[key].writeto(f"data/{value['ID'].decode('utf-8')}.fits.fz", overwrite=True)

0it [00:00, ?it/s]

2000it [00:15, 128.62it/s]


In [62]:
df['FWHM'].to_numpy()

array([0.        , 0.        , 0.        , ..., 0.00055736, 0.00044423,
       0.00029422], dtype=float32)